In [1]:
import pandas as pd
import gzip

In [2]:
from localpdb import PDB
from Bio.PDB.PDBParser import PDBParser
from Bio.PDB import PDBIO
from Bio.PDB.PDBIO import Select

In [3]:
SAMCC_PICKLE = 'pdb_scan_nr_topology_20190927.p' # SAMCC pickle location
OLIGOMERIZATION = 4 # oligomerization status to select
OUT_PATH = 'tetr_cc' # out path to save fragments of structures containing CC domians

#### Load localpdb and SAMCC pickle

In [4]:
lpdb = PDB(db_path='/home/db/localpdb', version='20190927', plugins=['PDBSeqresMapper'])
df = pd.read_pickle(SAMCC_PICKLE)

#### Optionally filter localpdb chains by sequence identity

In [5]:
lpdb.load_clustering_data(cutoff='90')
lpdb.structures = lpdb.structures[lpdb.structures['type'] == 'prot']
lpdb.structures = lpdb.structures[lpdb.structures['method'] == 'diffraction']
lpdb.chains = lpdb.chains.loc[lpdb.chains.groupby(by='blast-90')['resolution'].idxmin()]

#### Parse SAMCC pickle (select only entries that are present in localpdb - some chains are present in biounits but not in PDB data - but they are the same (symmetry) so we can skip them

In [6]:
data = {}
for key, value in df.iterrows():
    tmp_df = df.loc[key, 'data']
    topology_len = value['toplogy_len']
    pdb = value['pdb']
    antiparallel = any(value['toplogy'])
    bundle_id = value['bundle_id']
    # Iterate over each element in topology
    for i in range(0, topology_len):
        try:
            helix_df = tmp_df.xs(i, level='chain').sort_values(by='res_number')
            chain = helix_df['chain_name'].values[0]
            pdb_chain = '{}_{}'.format(pdb, chain)
            if pdb_chain in lpdb.chains.index:
                pdb_beg, pdb_end = helix_df['res_number'].values[0], helix_df['res_number'].values[-1]
                data[bundle_id] = (topology_len, antiparallel, pdb_beg, pdb_end, pdb_chain)
        except KeyError:
            pass
        except ValueError:
            pass

In [7]:
len(data), len(df)

(4298, 10893)

#### Dataset 

In [8]:
dataset = pd.DataFrame.from_dict(data, orient='index', columns=['oligo', 'ap', 'beg', 'end', 'pdb_chain'])

#### Dataset filtering - feel free to adjust according to your needs
###### Minimum sequence length is (I think) 11 - if you want to generate fragments of length 9

In [9]:
dataset = dataset[dataset['oligo'] == OLIGOMERIZATION]
dataset['seq_len'] = dataset.apply(lambda x: x.end - x.beg + 1, axis=1)
dataset = dataset[dataset['seq_len'] >= 14]

In [10]:
dataset

,oligo,ap,beg,end,pdb_chain,seq_len
4pxu_0,4,True,701,755,4pxu_A,55
6f63_0,4,True,678,755,6f63_A,78
3lbw_0,4,False,26,44,3lbw_A,19
1gl2_0,4,False,160,203,1gl2_D,44
6b87_0,4,True,58,95,6b87_A,38
...,...,...,...,...,...,...
6q5r_0,4,True,6,25,6q5r_A,20
6q5h_0,4,True,3,18,6q5h_A,16
6egc_0,4,True,121,147,6egc_A,27
4hb1_0,4,True,106,120,4hb1_A,15


In [11]:
class PDBExtractor(Select): 
    def __init__(self, start, end): 
        """Initialize the class.""" 
        self.start = start 
        self.end = end 
  
   
    def accept_residue(self, residue): 
        hetatm_flag, resseq, icode = residue.get_id()  
        if self.start <= resseq <= self.end: 
            return 1 
        return 0 

def cut(pdb_chain, beg, end, out_fn):
    fh = gzip.open('/home/db/localpdb/pdb_chain/{}/{}.pdb.gz'.format(pdb_chain[1:3], pdb_chain), mode='rt')
    structure = PDBParser().get_structure('s', fh)    
    io = PDBIO()
    io.set_structure(structure)
    io.save(out_fn, PDBExtractor(beg, end))

In [12]:
for key, value in dataset.iterrows():
    cut(value['pdb_chain'], value['beg'], value['end'], '{}/{}.pdb'.format(OUT_PATH, key))

/home/users/jludwiczak/venvs/work_gpu/lib/python3.6/site-packages/Bio/PDB/Atom.py:205: PDBConstructionWarning: Used element 'U' for Atom (name=UNK) with given element ''
  warnings.warn(msg, PDBConstructionWarning)


### Now run fragment picker (it'll pick fragments irrespective of sequence from the structure set we obtained from SAMCC and localpdb)

#### Fragments have to be generated for each sequence length independently (sequence_length), and if you want to use it with F&D runs you have to generate fragments of length (frag_length) 3 and 9

In [13]:
%%bash
ls tetr_cc/* > PDB_LIST

/opt/apps/rosetta-3.10/main/source/bin/struc_set_fragment_picker.linuxgccrelease -in:file:l PDB_LIST \
        -frags:n_frags 200 -struc_set_fragment_picker::frag_length 3 \
        -struc_set_fragment_picker::sequence_length 50 \
        -struc_set_fragment_picker::frag_name tetr_3.frag
        
        
/opt/apps/rosetta-3.10/main/source/bin/struc_set_fragment_picker.linuxgccrelease -in:file:l PDB_LIST \
        -frags:n_frags 200 -struc_set_fragment_picker::frag_length 9 \
        -struc_set_fragment_picker::sequence_length 50 \
        -struc_set_fragment_picker::frag_name tetr_9.frag

core.init: Checking for fconfig files in pwd and ./rosetta/flags 
core.init: Rosetta version: rosetta.source.release-188 r188 2018.33+release.7111c54 7111c54c14ba9a53c012a524f8f1438a8e3fb020 https://www.rosettacommons.org 2018-08-14T01:44:18.723947
core.init: command: /opt/apps/rosetta-3.10/main/source/bin/struc_set_fragment_picker.linuxgccrelease -in:file:l PDB_LIST -frags:n_frags 200 -struc_set_fragment_picker::frag_length 3 -struc_set_fragment_picker::sequence_length 50 -struc_set_fragment_picker::frag_name tetr_3.frag
core.init: 'RNG device' seed mode, using '/dev/urandom', seed=428906092 seed_offset=0 real_seed=428906092
core.init.random: RandomGenerator:init: Normal mode, seed=428906092 RG_type=mt19937
core.init: Resolved executable path: /opt/apps/rosetta-3.10/main/source/build/src/release/linux/4.15/64/x86/gcc/7/default/struc_set_fragment_picker.default.linuxgccrelease
core.init: Looking for database based on location of executable: /opt/apps/rosetta-3.10/main/database/
struc_s